तलका नोटबुकहरू चलाउनका लागि, यदि तपाईंले अझै नगरेको भए, तपाईंले `text-embedding-ada-002` लाई आधार मोडेलको रूपमा प्रयोग गर्ने मोडेल डिप्लोय गर्नुपर्छ र .env फाइल भित्र डिप्लोयमेन्ट नामलाई `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` को रूपमा सेट गर्नुपर्छ।


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

अर्को, हामी Pandas Dataframe मा Embedding Index लोड गर्न जाँदैछौं। Embedding Index `embedding_index_3m.json` नामक JSON फाइलमा संग्रहित छ। Embedding Index मा अक्टोबर 2023 को अन्त्यसम्मका प्रत्येक YouTube ट्रान्सक्रिप्टहरूको Embeddings समावेश छन्।


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

अर्को, हामी `get_videos` नामक एक फंक्शन बनाउनेछौं जुन क्वेरीको लागि Embedding Index मा खोजी गर्नेछ। फंक्शनले क्वेरीसँग सबैभन्दा मिल्दोजुल्दो शीर्ष ५ भिडियोहरू फर्काउनेछ। फंक्शन यसरी काम गर्छ:

1. पहिले, Embedding Index को एक प्रतिलिपि बनाइन्छ।
2. त्यसपछि, OpenAI Embedding API प्रयोग गरेर क्वेरीको Embedding गणना गरिन्छ।
3. त्यसपछि Embedding Index मा `similarity` नामक नयाँ स्तम्भ बनाइन्छ। `similarity` स्तम्भमा क्वेरी Embedding र प्रत्येक भिडियो सेगमेन्टको Embedding बीचको कोसाइन समानता हुन्छ।
4. त्यसपछि, Embedding Index लाई `similarity` स्तम्भद्वारा फिल्टर गरिन्छ। Embedding Index लाई केवल ती भिडियोहरू समावेश गर्न फिल्टर गरिन्छ जसको कोसाइन समानता ०.७५ वा सो भन्दा बढी हुन्छ।
5. अन्तमा, Embedding Index लाई `similarity` स्तम्भद्वारा क्रमबद्ध गरिन्छ र शीर्ष ५ भिडियोहरू फर्काइन्छ।


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

यो फङ्क्शन धेरै सरल छ, यसले खोजी सोधपुछको परिणामहरू मात्र प्रिन्ट गर्छ।


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. पहिलो, Embedding Index लाई Pandas Dataframe मा लोड गरिन्छ।
2. त्यसपछि, प्रयोगकर्तालाई क्वेरी प्रविष्ट गर्न अनुरोध गरिन्छ।
3. त्यसपछि `get_videos` फंक्शनलाई क्वेरीको लागि Embedding Index खोज्न कल गरिन्छ।
4. अन्तमा, `display_results` फंक्शनलाई प्रयोगकर्तालाई परिणामहरू देखाउन कल गरिन्छ।
5. त्यसपछि प्रयोगकर्तालाई अर्को क्वेरी प्रविष्ट गर्न अनुरोध गरिन्छ। यो प्रक्रिया तबसम्म जारी रहन्छ जबसम्म प्रयोगकर्ताले `exit` प्रविष्ट गर्दैन।

![](../../../../translated_images/ne/notebook-search.1e320b9c7fcbb0bc.png)

तपाईंलाई क्वेरी प्रविष्ट गर्न अनुरोध गरिनेछ। एउटा क्वेरी प्रविष्ट गर्नुहोस् र इन्टर थिच्नुहोस्। एप्लिकेशनले क्वेरीसँग सम्बन्धित भिडियोहरूको सूची फर्काउनेछ। एप्लिकेशनले प्रश्नको उत्तर रहेको भिडियोको स्थानमा लिंक पनि फर्काउनेछ।

यहाँ केही क्वेरीहरू प्रयास गर्नका लागि छन्:

- Azure Machine Learning के हो?
- convolutional neural networks कसरी काम गर्छन्?
- neural network के हो?
- के म Azure Machine Learning सँग Jupyter Notebooks प्रयोग गर्न सक्छु?
- ONNX के हो?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
यो दस्तावेज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताका लागि प्रयासरत छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धता हुन सक्छ। मूल दस्तावेज यसको मूल भाषामा नै अधिकारिक स्रोत मानिनु पर्छ। महत्वपूर्ण जानकारीका लागि व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
